In [ ]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Point
import mediapipe as mp
from ultralytics import YOLO
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import bluetooth
import asyncio
from openpyxl import load_workbook  # For Excel file manipulation
import time

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\Users\alie0\OneDrive\Desktop\to rokaia\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to connect to the webcam")
else:
    print("Connected to the webcam successfully")

# Initialize YOLO model
model = YOLO(r"C:\Users\alie0\OneDrive\Desktop\to rokaia\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize MediaPipe Hands
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

def send_data(msg, delay=2):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
                print(f"Message sent: {msg.decode('utf-8')}")
                time.sleep(delay)  # Introduce a delay between messages
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag
    with open('gesture_recognizer1.pkl', 'rb') as file:
        recognizer = pickle.load(file)
    
    points = []
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()

            # Convert the frame to RGB for MediaPipe processing
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            annotated_image = frame.copy()

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    # Capture important hand landmarks for gesture recognition
                    wrist = Point(hand_landmarks.landmark[0].x, hand_landmarks.landmark[0].y, 1)
                    thumb_cmc = Point(hand_landmarks.landmark[1].x, hand_landmarks.landmark[1].y, 1)
                    thumb_mcp = Point(hand_landmarks.landmark[2].x, hand_landmarks.landmark[2].y, 1)
                    thumb_ip = Point(hand_landmarks.landmark[3].x, hand_landmarks.landmark[3].y, 1)
                    thumb_tip = Point(hand_landmarks.landmark[4].x, hand_landmarks.landmark[4].y, 1)

                    points = [wrist, thumb_cmc, thumb_mcp, thumb_ip, thumb_tip]

                    print(f"Hand landmarks: {points}")  # Debug: Log the landmarks
                    try:
                        # Check if the recognizer is working and making a prediction
                        prediction = recognizer.recognize(points)
                        print(f"Predictions: {prediction}")  # Debugging
                        
                        if prediction and len(prediction) > 0:
                            detected_gesture = prediction[0]
                            print(f"Detected gesture: {detected_gesture}")  # Debugging
                            msg_pred = f"{detected_gesture}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)  # Send data to the client
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred:", str(e))
            else:
                print("No hand landmarks detected.")

            # Display the annotated image with landmarks
            cv2.imshow('Camera - Gesture Recognition', annotated_image)

            # Exit if the 'q' key is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break
            
            
            
            
            
#  face recongnition part 
excel_file_face = r"C:\Users\alie0\OneDrive\Desktop\to rokaia\Smart_Shopping-main\UserProfileByFace.xlsx"
columns = ["Name", "quantity", "products", "emotion"]
# Check if the file exists; if not, create it
try:
    df = pd.read_excel(excel_file_face)
except FileNotFoundError:
    df = pd.DataFrame(columns=columns)
    df.to_excel(excel_file_face, index=False)
def update_excel(name, emotion):
    global df
    # Check if the user already exists in the sheet
    new_row = {"Name": name, "quantity": 1, "products": "None", "emotion": emotion}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.to_excel(excel_file_face, index=False)
def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                    update_excel(name, emotion)
                except Exception as e:
                    emotion = "Unknown"
                
                # Update the Excel sheet with the user's data
                
                
                display_text = f"{name} is {emotion}"
                send_data(display_text.encode("utf-8"))
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Bluetooth function using asyncio

async def continuous_bluetooth_login(scan_duration=3, cycles=1):
    print(f"Starting continuous scanning for Bluetooth devices (each scan lasts {scan_duration} seconds)...")

    try:
        while True:  # Continuous scanning loop
            print("Scanning for Bluetooth devices...")
            all_devices = []  # List to collect all devices detected in the current cycle

            # Perform multiple scan cycles per round
            for _ in range(cycles):
                nearby_devices = bluetooth.discover_devices(
                    duration=scan_duration, lookup_names=True, device_id=-1, flush_cache=True
                )
                for addr, name in nearby_devices:
                    all_devices.append((addr, name))

            # Remove duplicates based on MAC address
            unique_devices = {device[0]: device[1] for device in all_devices}.items()

            if not unique_devices:
                print("No devices found during this scan.")
            else:
                for mac_address, device_name in unique_devices:
                    print(f"Detected device: {device_name}, MAC Address: {mac_address}")

                    if mac_address in registered_macs:
                        user_data = mac_df.loc[mac_df["mac_address"] == mac_address]
                        user = user_data.iloc[0]  # Get the matched user profile
                        username = user['Name']
                        role = user['role']

                        # Format the message to send to the C# application
                        msg = f"{username} has successfully logged in. Role: {role}".encode("utf-8")
                        send_data(msg)  # Send the message to the C# app via socket
                        print(f"Login successful for user: {username}, Role: {role}")
                    else:
                        print(f"Unregistered device detected: {mac_address} (Name: {device_name}).")

            await asyncio.sleep(5)  # Pause before the next scan cycle
    except Exception as e:
        print(f"An error occurred during Bluetooth scanning: {e}")


def scan_in_background():
    try:
        asyncio.run(continuous_bluetooth_login(scan_duration=5))
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            asyncio.ensure_future(continuous_bluetooth_login(scan_duration=5))


# Start threads
camera_thread = threading.Thread(target=capture_frame)
face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
object_detection_thread = threading.Thread(target=detect_objects)
scanner_thread = threading.Thread(target=scan_in_background)
camera_thread.start()
face_emotion_thread.start()
gesture_thread.start()
object_detection_thread.start()
scanner_thread.start()
camera_thread.join()
face_emotion_thread.join()
gesture_thread.join()
object_detection_thread.join()
scanner_thread.join()
cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()


Device connected
Connected to the webcam successfully
10 encoding images found.
Encoding images loaded
Starting continuous scanning for Bluetooth devices (each scan lasts 5 seconds)...
Scanning for Bluetooth devices...
No devices found during this scan.
Message sent: Rawan is neutral
Scanning for Bluetooth devices...
No devices found during this scan.
Message sent: Rawan is neutral
Scanning for Bluetooth devices...
No devices found during this scan.
Message sent: Rawan is neutral
Message sent: Rawan is sad
Scanning for Bluetooth devices...
No devices found during this scan.
Message sent: Rawan is neutral
Message sent: Unknown is neutral
Scanning for Bluetooth devices...
No devices found during this scan.
Message sent: Rawan is surprise
Message sent: Rawan is neutral
Scanning for Bluetooth devices...
No devices found during this scan.
Message sent: Rawan is happy
Message sent: Rawan is happy
Scanning for Bluetooth devices...
No devices found during this scan.
Message sent: Rawan is hap

In [6]:
import socket
import cv2
import pickle
import threading
from queue import Queue
from dollarpy import Recognizer, Template, Point  # Ensure Recognizer is imported
import mediapipe as mp
from ultralytics import YOLO
from deepface import DeepFace
from simple_facerec import SimpleFacerec
import pandas as pd
import bluetooth
import asyncio
from openpyxl import load_workbook  # For Excel file manipulation
import time
import os
import numpy as np

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

HOSTNAME = '127.0.0.1'
PORT = 5010

# Path to the Excel file
excel_path = r"C:\Users\alie0\OneDrive\Desktop\to rokaia\Smart_Shopping-main\bloutooth.xlsx"

# Load registered MAC addresses from Excel
try:
    mac_df = pd.read_excel(excel_path)
    registered_macs = set(mac_df['mac_address'].dropna().astype(str).tolist())
except Exception as e:
    print("Error reading Excel file:", e)
    registered_macs = set()  # Start with an empty set if there’s an issue

# Socket setup
soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
soc.bind((HOSTNAME, PORT))
soc.listen(5)
conn, addr = soc.accept()
print("Device connected")

# Initialize the camera
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Unable to connect to the webcam")
else:
    print("Connected to the webcam successfully")

# Initialize YOLO model
model = YOLO(r"C:\Users\alie0\OneDrive\Desktop\to rokaia\Smart_Shopping-main\best (2).pt")
target_class_names = ["deodorant", "vitamin"]
target_classes = [index for index, name in model.names.items() if name in target_class_names]

# Initialize face recognition
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Lock for thread synchronization
camera_lock = threading.Lock()
socket_lock = threading.Lock()

# Frame queue for threads
frame_queue = Queue(maxsize=10)

# Shutdown flag
shutdown_flag = False

def send_data(msg, delay=2):
    try:
        with socket_lock:
            if conn.fileno() != -1:
                conn.send(msg)
                print(f"Message sent: {msg.decode('utf-8')}")
                time.sleep(delay)  # Introduce a delay between messages
            else:
                print("Socket is closed.")
    except socket.error as e:
        print(f"Socket error: {e}")
        global shutdown_flag
        shutdown_flag = True
        conn.close()

def capture_frame():
    global shutdown_flag
    while not shutdown_flag:
        with camera_lock:
            ret, frame = cap.read()
            if ret:
                if frame_queue.full():
                    frame_queue.get()
                frame_queue.put(frame)
            else:
                print("Failed to capture frame.")
                shutdown_flag = True
                break

def loadTemplates(save_path):
    if os.path.exists(save_path):
        with open(save_path, 'rb') as f:
            return pickle.load(f)
    return []

def getPointsFromFrame(frame):
    points = []

    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image)

        # Process hand landmarks
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                for landmark in hand_landmarks.landmark:
                    x, y = landmark.x, landmark.y
                    if not (np.isnan(x) or np.isnan(y)):
                        points.append(Point(x, y, stroke_id=0))
    return points

def capture_gestures():
    global shutdown_flag  # Ensure we're modifying the global shutdown_flag

    # Load gesture templates
    template_file = "gesture_templates2.pkl"
    templates = loadTemplates(template_file)

    if not templates:
        print("No templates loaded. Ensure the gesture_templates2.pkl file exists and contains valid data.")
        return

    # Initialize Recognizer with loaded templates
    recognizer = Recognizer(templates)

    with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
        while not shutdown_flag:
            if not frame_queue.empty():
                frame = frame_queue.get()

                # Process frame for hand landmarks
                points = getPointsFromFrame(frame)
                x_values = [p.x for p in points]
                y_values = [p.y for p in points]

                if points and (max(x_values) - min(x_values) != 0) and (max(y_values) - min(y_values) != 0):
                    try:
                        # Recognize gesture
                        result = recognizer.recognize(points)

                        if result:
                            best_match = result[0]
                            gesture_name = best_match

                            # Display detected gesture on the frame
                            cv2.putText(frame, f"Gesture: {gesture_name}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                            # Send detected gesture via socket
                            msg_pred = f"Detected Gesture: {gesture_name}".encode("utf-8")
                            if conn.fileno() != -1:  # Check if the connection is still open
                                send_data(msg_pred)
                            else:
                                print("Connection closed, unable to send gesture data.")
                    except Exception as e:
                        print("An error occurred during gesture recognition:", str(e))
                else:
                    cv2.putText(frame, "No gesture detected", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                # Display the frame with annotations
                cv2.imshow("Gesture Recognition", frame)

                # Exit if the 'q' key is pressed
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    shutdown_flag = True
                    break
excel_file_face = r"C:\Users\alie0\OneDrive\Desktop\to rokaia\Smart_Shopping-main\UserProfileByFace.xlsx"
columns = ["Name", "quantity", "products", "emotion"]
# Check if the file exists; if not, create it
try:
    df = pd.read_excel(excel_file_face)
except FileNotFoundError:
    df = pd.DataFrame(columns=columns)
    df.to_excel(excel_file_face, index=False)
def update_excel(name, emotion):
    global df
    # Check if the user already exists in the sheet
    new_row = {"Name": name, "quantity": 1, "products": "None", "emotion": emotion}
    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    df.to_excel(excel_file_face, index=False)
def detect_face_and_emotions():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            flipped_frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2RGB)
            face_locations, face_names = sfr.detect_known_faces(flipped_frame)

            for face_loc, name in zip(face_locations, face_names):
                y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
                face_roi = rgb_frame[y1:y2, x1:x2]
                try:
                    emotion_result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                    emotion = emotion_result[0]['dominant_emotion']
                    update_excel(name, emotion)
                except Exception as e:
                    emotion = "Unknown"
                
                # Update the Excel sheet with the user's data
                
                
                display_text = f"{name} is {emotion}"
                send_data(display_text.encode("utf-8"))
                cv2.putText(flipped_frame, display_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 200), 2)
                cv2.rectangle(flipped_frame, (x1, y1), (x2, y2), (0, 0, 200), 2)

            cv2.imshow("Face Recognition and Emotion Detection", flipped_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

def detect_objects():
    global shutdown_flag
    while not shutdown_flag:
        if not frame_queue.empty():
            frame = frame_queue.get()
            frame_resized = cv2.resize(frame, (640, 640))
            results = model(frame_resized, conf=0.9, classes=target_classes)

            annotated_frame = results[0].plot()
            if len(results[0].boxes.cls) > 0:
                for box, score, cls in zip(results[0].boxes.xywh, results[0].boxes.conf, results[0].boxes.cls):
                    class_name = model.names[int(cls)]
                    confidence = float(score)
                    if class_name in target_class_names:
                        msg_class = f"Detected Object: {class_name}".encode("utf-8")
                        send_data(msg_class)

            cv2.imshow("Object Detection", annotated_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                shutdown_flag = True
                break

# Bluetooth function using asyncio

async def continuous_bluetooth_login(scan_duration=3, cycles=1):
    print(f"Starting continuous scanning for Bluetooth devices (each scan lasts {scan_duration} seconds)...")

    try:
        while True:  # Continuous scanning loop
            print("Scanning for Bluetooth devices...")
            all_devices = []  # List to collect all devices detected in the current cycle

            # Perform multiple scan cycles per round
            for _ in range(cycles):
                nearby_devices = bluetooth.discover_devices(
                    duration=scan_duration, lookup_names=True, device_id=-1, flush_cache=True
                )
                for addr, name in nearby_devices:
                    all_devices.append((addr, name))

            # Remove duplicates based on MAC address
            unique_devices = {device[0]: device[1] for device in all_devices}.items()

            if not unique_devices:
                print("No devices found during this scan.")
            else:
                for mac_address, device_name in unique_devices:
                    print(f"Detected device: {device_name}, MAC Address: {mac_address}")

                    if mac_address in registered_macs:
                        user_data = mac_df.loc[mac_df["mac_address"] == mac_address]
                        user = user_data.iloc[0]  # Get the matched user profile
                        username = user['Name']
                        role = user['role']

                        # Format the message to send to the C# application
                        msg = f"{username} has successfully logged in. Role: {role}".encode("utf-8")
                        send_data(msg)  # Send the message to the C# app via socket
                        print(f"Login successful for user: {username}, Role: {role}")
                    else:
                        print(f"Unregistered device detected: {mac_address} (Name: {device_name}).")

            await asyncio.sleep(5)  # Pause before the next scan cycle
    except Exception as e:
        print(f"An error occurred during Bluetooth scanning: {e}")


def scan_in_background():
    try:
        asyncio.run(continuous_bluetooth_login(scan_duration=5))
    except RuntimeError as e:
        if "This event loop is already running" in str(e):
            asyncio.ensure_future(continuous_bluetooth_login(scan_duration=5))


# Start threads
camera_thread = threading.Thread(target=capture_frame)
#face_emotion_thread = threading.Thread(target=detect_face_and_emotions)
gesture_thread = threading.Thread(target=capture_gestures)
#object_detection_thread = threading.Thread(target=detect_objects)
#scanner_thread = threading.Thread(target=scan_in_background)
camera_thread.start()
#face_emotion_thread.start()
gesture_thread.start()
#object_detection_thread.start()
#scanner_thread.start()
#camera_thread.join()
#face_emotion_thread.join()
gesture_thread.join()
#object_detection_thread.join()
#scanner_thread.join()
cap.release()
cv2.destroyAllWindows()
conn.send(bytes("q", "utf-8"))
conn.close()
soc.close()
# The rest of the face recognition, object detection, and Bluetooth scanning functions remain unchanged
# Add the necessary threading and thread join logic as shown in the original script

# Start threads



Device connected
Connected to the webcam successfully
10 encoding images found.
Encoding images loaded
Message sent: Detected Gesture: right
Message sent: Detected Gesture: right
Message sent: Detected Gesture: right
Message sent: Detected Gesture: right
Message sent: Detected Gesture: right
Message sent: Detected Gesture: right
Message sent: Detected Gesture: left
Message sent: Detected Gesture: left
Message sent: Detected Gesture: left
Message sent: Detected Gesture: left
Message sent: Detected Gesture: left
Message sent: Detected Gesture: left
Message sent: Detected Gesture: left
